In [1]:
# dependencies and setup
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime as dt
import numpy as np
import sqlalchemy
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func, ForeignKey
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Date, Integer, String, Float

In [2]:
# files to load
file1 = "Raw_Data/IMDb movies.csv"
file2 = "Raw_Data/rotten_tomatoes_movies.csv"
file3 = "Raw_Data/IMDb title_principals.csv"
file4 = "Raw_Data/IMDb names.csv"

# read files and store into pandas dataframe
IMDB_movies = pd.read_csv(file1, low_memory=False)
rotten_tomatoes = pd.read_csv(file2, low_memory=False)
cast_and_crew = pd.read_csv(file3, low_memory=False)
actors = pd.read_csv(file4, low_memory=False)

In [3]:
# view IMDB_movies dataframe
IMDB_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [4]:
# rename IMDB avg_vote to ratings
IMDB_movies = IMDB_movies.rename(columns={"avg_vote":"imdb_ratings", "worlwide_gross_income":"worldwide_gross_income"})
IMDB_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,imdb_ratings,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [5]:
# remove metascore, reviews_from_users, and reviews_from_critics from IMDB
IMDB_movies = IMDB_movies.drop(columns=['metascore', 'reviews_from_users', 'reviews_from_critics'])
IMDB_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,imdb_ratings,votes,budget,usa_gross_income,worldwide_gross_income
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN


In [6]:
#Look at the data types of all the columns
IMDB_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   imdb_title_id           85855 non-null  object 
 1   title                   85855 non-null  object 
 2   original_title          85855 non-null  object 
 3   year                    85855 non-null  object 
 4   date_published          85855 non-null  object 
 5   genre                   85855 non-null  object 
 6   duration                85855 non-null  int64  
 7   country                 85791 non-null  object 
 8   language                85022 non-null  object 
 9   director                85768 non-null  object 
 10  writer                  84283 non-null  object 
 11  production_company      81400 non-null  object 
 12  actors                  85786 non-null  object 
 13  description             83740 non-null  object 
 14  imdb_ratings            85855 non-null

In [7]:
#Checks max lenghts of all string i.e. object columns
for c in IMDB_movies:
    if IMDB_movies[c].dtype=='object':
        print(f'{c} {IMDB_movies[c].str.len().max()}')

imdb_title_id 10
title 196
original_title 196
year 13
date_published 13
genre 31
country 225.0
language 163.0
director 62.0
writer 64.0
production_company 101.0
actors 415.0
description 402.0
budget 16.0
usa_gross_income 12.0
worldwide_gross_income 14.0


In [8]:
#Year should be integer, checking which values are forcing pandas to make it string i.e. object
IMDB_movies['year'].unique()

array(['1894', '1906', '1911', '1912', '1919', '1913', '1914', '1915',
       '1916', '1917', '1918', '1920', '1921', '1924', '1922', '1923',
       '1925', '1926', '1935', '1927', '1928', '1983', '1929', '1930',
       '1932', '1931', '1937', '1938', '1933', '1934', '1936', '1940',
       '1939', '1942', '1943', '1941', '1948', '1944', '2001', '1946',
       '1945', '1947', '1973', '1949', '1950', '1952', '1951', '1962',
       '1953', '1954', '1955', '1961', '1956', '1958', '1957', '1959',
       '1960', '1963', '1965', '1971', '1964', '1966', '1968', '1967',
       '1969', '1976', '1970', '1979', '1972', '1981', '1978', '2000',
       '1989', '1975', '1974', '1986', '1990', '2018', '1977', '1982',
       '1980', '1993', '1984', '1985', '1988', '1987', '2005', '1991',
       '2002', '1994', '1992', '1995', '2017', '1997', '1996', '2006',
       '1999', '1998', '2007', '2008', '2003', '2004', '2010', '2009',
       '2011', '2013', '2012', '2016', '2015', '2014', '2019', '2020',
      

In [9]:
#Replacing 'TV Movie 2019' to '2019'
IMDB_movies['year']= IMDB_movies['year'].replace(['TV Movie 2019'],2019)

In [10]:
#Data After Replacing
IMDB_movies['year'].unique()

array(['1894', '1906', '1911', '1912', '1919', '1913', '1914', '1915',
       '1916', '1917', '1918', '1920', '1921', '1924', '1922', '1923',
       '1925', '1926', '1935', '1927', '1928', '1983', '1929', '1930',
       '1932', '1931', '1937', '1938', '1933', '1934', '1936', '1940',
       '1939', '1942', '1943', '1941', '1948', '1944', '2001', '1946',
       '1945', '1947', '1973', '1949', '1950', '1952', '1951', '1962',
       '1953', '1954', '1955', '1961', '1956', '1958', '1957', '1959',
       '1960', '1963', '1965', '1971', '1964', '1966', '1968', '1967',
       '1969', '1976', '1970', '1979', '1972', '1981', '1978', '2000',
       '1989', '1975', '1974', '1986', '1990', '2018', '1977', '1982',
       '1980', '1993', '1984', '1985', '1988', '1987', '2005', '1991',
       '2002', '1994', '1992', '1995', '2017', '1997', '1996', '2006',
       '1999', '1998', '2007', '2008', '2003', '2004', '2010', '2009',
       '2011', '2013', '2012', '2016', '2015', '2014', '2019', '2020',
      

In [11]:
#Convert Year to Integer from Object i.e. string
IMDB_movies['year']=pd.to_numeric(IMDB_movies['year'])

In [12]:
#verify after conversion
IMDB_movies['year'].dtype

dtype('int64')

In [13]:
#Date Published should be date, checking which values are forcing pandas to make it string i.e. object
IMDB_movies['date_published'].unique()
#Looking at it typically all values are of length 10 (yyyy-mm-dd)

array(['1894-10-09', '1906-12-26', '1911-08-19', ..., '2020-10-22',
       '2019-01-13', '2020-09-04'], dtype=object)

In [14]:
#Finding the value that has length greater than 10
[x for x in IMDB_movies['date_published'] if len(x) > 10]

['TV Movie 2019']

In [15]:
#Finding the movie with publised data of 'TV Movie 2019'
IMDB_movies[IMDB_movies['date_published']=='TV Movie 2019']

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,imdb_ratings,votes,budget,usa_gross_income,worldwide_gross_income
83917,tt8206668,Bad Education,Bad Education,2019,TV Movie 2019,"Biography, Comedy, Crime",108,USA,English,Cory Finley,"Mike Makowsky, Robert Kolker",HBO Films,"Hugh Jackman, Ray Romano, Welker White, Alliso...",The beloved superintendent of New York's Rosly...,7.1,23973,NaN,NaN,NaN


In [16]:
#By google search release date of "Bad Education" was 8th Sept 2019
#Replacing the date to 8th Sept 2019
IMDB_movies['date_published']= IMDB_movies['date_published'].replace(['TV Movie 2019'],'2019-09-08')

In [17]:
#Convert date_published to Date from Object i.e. string
IMDB_movies['date_published']=pd.to_datetime(IMDB_movies['date_published'],format='%Y-%m-%d')

In [18]:
#Verify datatype after conversion
IMDB_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   imdb_title_id           85855 non-null  object        
 1   title                   85855 non-null  object        
 2   original_title          85855 non-null  object        
 3   year                    85855 non-null  int64         
 4   date_published          85855 non-null  datetime64[ns]
 5   genre                   85855 non-null  object        
 6   duration                85855 non-null  int64         
 7   country                 85791 non-null  object        
 8   language                85022 non-null  object        
 9   director                85768 non-null  object        
 10  writer                  84283 non-null  object        
 11  production_company      81400 non-null  object        
 12  actors                  85786 non-null  object

In [19]:
# view Rotten Tomatoes dataframe
rotten_tomatoes.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2/12/2010,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",4/30/2010,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",10/5/1979,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",4/13/1957,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1/1/1954,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


In [20]:
# view columns and datatypes of rotten_tomatoes
rotten_tomatoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17712 entries, 0 to 17711
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   rotten_tomatoes_link              17712 non-null  object 
 1   movie_title                       17712 non-null  object 
 2   movie_info                        17391 non-null  object 
 3   critics_consensus                 9134 non-null   object 
 4   content_rating                    17712 non-null  object 
 5   genres                            17693 non-null  object 
 6   directors                         17518 non-null  object 
 7   authors                           16170 non-null  object 
 8   actors                            17360 non-null  object 
 9   original_release_date             16546 non-null  object 
 10  streaming_release_date            17328 non-null  object 
 11  runtime                           17398 non-null  float64
 12  prod

In [21]:
# Original release date should be date, checking which values are forcing pandas to make it string i.e. object
rotten_tomatoes['original_release_date'].unique()
#Looking at it typically all values are of length 10 (mm/dd/yyyy)

array(['2/12/2010', '4/30/2010', '10/5/1979', ..., '10/2/1981',
       '12/17/1964', '6/17/1964'], dtype=object)

In [22]:
# Convert original_release_date to Date from Object i.e. string
rotten_tomatoes['original_release_date']=pd.to_datetime(rotten_tomatoes['original_release_date'],format='%m/%d/%Y')

In [23]:
# Convert streaming_release_date to Date from Object i.e. string
rotten_tomatoes['streaming_release_date']=pd.to_datetime(rotten_tomatoes['streaming_release_date'],format='%m/%d/%Y')

In [24]:
# view columns and datatypes of rotten_tomatoes to see that dates changed to datetime64[ns]
rotten_tomatoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17712 entries, 0 to 17711
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   rotten_tomatoes_link              17712 non-null  object        
 1   movie_title                       17712 non-null  object        
 2   movie_info                        17391 non-null  object        
 3   critics_consensus                 9134 non-null   object        
 4   content_rating                    17712 non-null  object        
 5   genres                            17693 non-null  object        
 6   directors                         17518 non-null  object        
 7   authors                           16170 non-null  object        
 8   actors                            17360 non-null  object        
 9   original_release_date             16546 non-null  datetime64[ns]
 10  streaming_release_date            17328 non-nu

In [25]:
# merge IMDB_movies and rotten_tomatoes together
merged_movies=IMDB_movies.merge(rotten_tomatoes, how='left', left_on=['original_title','date_published'], right_on=['movie_title','original_release_date'],suffixes=('_imdb','_rt'))
merged_movies.head(20)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,production_company_rt,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913-01-01,"Biography, Drama",60,USA,English,Sidney Olcott,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,tt0002423,Madame DuBarry,Madame DuBarry,1919,1919-11-26,"Biography, Drama, Romance",85,Germany,German,Ernst Lubitsch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,tt0002445,Quo Vadis?,Quo Vadis?,1913,1913-03-01,"Drama, History",120,Italy,Italian,Enrico Guazzoni,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,tt0002452,Independenta Romaniei,Independenta Romaniei,1912,1912-09-01,"History, War",120,Romania,NaN,"Aristide Demetriade, Grigore Brezeanu",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,tt0002461,Richard III,Richard III,1912,1912-10-15,Drama,55,"France, USA",English,"André Calmettes, James Keane",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# see rows in IMDB_movies
len(IMDB_movies)

85855

In [27]:
# see rows in rotten_tomatoes
len(rotten_tomatoes)

17712

In [28]:
# see rows in merged_movies
len(merged_movies)

85855

In [29]:
# see just the ratings of merged file
Ratings = merged_movies[['title','year','original_release_date','imdb_ratings','tomatometer_rating','audience_rating','imdb_title_id']]
Ratings.head()

,title,year,original_release_date,imdb_ratings,tomatometer_rating,audience_rating,imdb_title_id
0,Miss Jerry,1894,NaT,5.9,NaN,NaN,tt0000009
1,The Story of the Kelly Gang,1906,NaT,6.1,NaN,NaN,tt0000574
2,Den sorte drøm,1911,NaT,5.8,NaN,NaN,tt0001892
3,Cleopatra,1912,NaT,5.2,NaN,NaN,tt0002101
4,L'Inferno,1911,NaT,7.0,NaN,NaN,tt0002130


In [30]:
# Create a rotten tomatoes year columnn based on the original release date
Ratings['rt_year'] = pd.DatetimeIndex(Ratings['original_release_date']).year

# Fill in null values of original release date column (now rt_year) with 0 and change to integer
Ratings['rt_year'] = Ratings['rt_year'].fillna(0)
Ratings['rt_year'] = Ratings['rt_year'].astype(int)

# remove original_release_date column
del Ratings['original_release_date']
Ratings.head()

,title,year,imdb_ratings,tomatometer_rating,audience_rating,imdb_title_id,rt_year
0,Miss Jerry,1894,5.9,NaN,NaN,tt0000009,0
1,The Story of the Kelly Gang,1906,6.1,NaN,NaN,tt0000574,0
2,Den sorte drøm,1911,5.8,NaN,NaN,tt0001892,0
3,Cleopatra,1912,5.2,NaN,NaN,tt0002101,0
4,L'Inferno,1911,7.0,NaN,NaN,tt0002130,0


In [31]:
# creating clean ratings DF where IMDB movies and Rotten Tomatoes movies match in year
clean_ratings = Ratings.loc[(Ratings['year']== Ratings['rt_year'])]

In [32]:
# see rows in clean_ratings 
len(clean_ratings)

2197

In [33]:
clean_ratings.head()

,title,year,imdb_ratings,tomatometer_rating,audience_rating,imdb_title_id,rt_year
277,Sangue e arena,1922,6.4,100.0,61.0,tt0012952,1922
302,Robin Hood,1922,7.2,100.0,72.0,tt0013556,1922
473,Il pirata nero,1926,7.1,100.0,68.0,tt0016654,1926
488,La carne e il diavolo,1926,7.6,93.0,84.0,tt0016884,1926
565,Cosetta,1927,7.3,87.0,81.0,tt0018033,1927


In [34]:
# view columns and datatypes of clean_ratings
clean_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2197 entries, 277 to 85774
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               2197 non-null   object 
 1   year                2197 non-null   int64  
 2   imdb_ratings        2197 non-null   float64
 3   tomatometer_rating  2191 non-null   float64
 4   audience_rating     2184 non-null   float64
 5   imdb_title_id       2197 non-null   object 
 6   rt_year             2197 non-null   int32  
dtypes: float64(3), int32(1), int64(1), object(2)
memory usage: 128.7+ KB


In [35]:
# see rows in movie_names_csv
len(cast_and_crew)

835513

In [36]:
# view columns and datatypes of movie_names_csv
cast_and_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835513 entries, 0 to 835512
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   imdb_title_id  835513 non-null  object
 1   ordering       835513 non-null  int64 
 2   imdb_name_id   835513 non-null  object
 3   category       835513 non-null  object
 4   job            212731 non-null  object
 5   characters     340836 non-null  object
dtypes: int64(1), object(5)
memory usage: 38.2+ MB


In [37]:
# see rows in names_csv
len(actors)

297705

In [38]:
# view names_csv dataframe
actors.head()

,imdb_name_id,name,birth_name,height,bio,birth_details,date_of_birth,place_of_birth,death_details,date_of_death,place_of_death,reason_of_death,spouses_string,spouses,divorces,spouses_with_children,children
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987-06-22,"Los Angeles, California, USA",pneumonia,Robyn Smith (27 June 1980 - 22 June 1987) (hi...,2,0,1,2
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014-08-12,"New York City, New York, USA",stroke,Jason Robards (4 July 1961 - 10 September 196...,2,1,2,3
2,nm0000003,Brigitte Bardot,Brigitte Bardot,166.0,"Brigitte Bardot was born on September 28, 1934...","September 28, 1934 in Paris, France",1934-09-28,"Paris, France",NaN,NaN,NaN,NaN,Bernard d'Ormale (16 August 1992 - present)\n...,4,3,1,1
3,nm0000004,John Belushi,John Adam Belushi,170.0,"John Belushi was born in Chicago, Illinois, US...","January 24, 1949 in Chicago, Illinois, USA",1949-01-24,"Chicago, Illinois, USA","March 5, 1982 in Hollywood, Los Angeles, Calif...",1982-03-05,"Hollywood, Los Angeles, California, USA",acute cocaine and heroin intoxication,Judith Belushi-Pisano (31 December 1976 - 5 M...,1,0,0,0
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,179.0,"Ernst Ingmar Bergman was born July 14, 1918, t...","July 14, 1918 in Uppsala, Uppsala län, Sweden",1918-07-14,"Uppsala, Uppsala län, Sweden","July 30, 2007 in Fårö, Gotlands län, Sweden (...",2007-07-30,"Fårö, Gotlands län, Sweden",natural causes,Ingrid Bergman (11 November 1971 - 20 May 199...,5,4,5,8


In [39]:
# Checks max lengths of all string i.e. object columns
for c in actors:
    if actors[c].dtype=='object':
        print(f'{c} {actors[c].str.len().max()}')

imdb_name_id 10
name 54
birth_name 217
bio 40729.0
birth_details 184.0
date_of_birth 105.0
place_of_birth 167.0
death_details 171.0
date_of_death 94.0
place_of_death 112.0
reason_of_death 119.0
spouses_string 585.0


In [40]:
# If date_of_birth value is nan or length is >10 then defualt to NaN.
actors['date_of_birth'] = [ np.nan if pd.isnull(x) or len(x)>10  else x for x in actors['date_of_birth'] ]

In [41]:
# Verify Max length of the column is 10
actors['date_of_birth'].dropna().str.len().max()

10

In [42]:
# Convert DOB column to datetime
# There is one DOB of value '1564-04-23'. Pandas only recognizes dates >= '1677-09-21' (pd.Timestamp.min).
# errors='coerce' changes it outbound date values to NaT
actors['date_of_birth']=pd.to_datetime(actors['date_of_birth'],format='%Y-%m-%d',errors = 'coerce')

In [43]:
# show pandas minimum date
pd.Timestamp.min

Timestamp('1677-09-21 00:12:43.145225')

In [44]:
# show pandas maximum date
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

In [45]:
# Date of death values with length grater than 10
[x for x in actors['date_of_death'].dropna() if len(x)>10]

['2017 in\xa0Québec, Canada',
 '2001 in\xa0Spain',
 '1999 in\xa0Rome, Lazio, Italy',
 '1972 in\xa0Beirut, Lebanon',
 '2006 in\xa0Chiltern, Buckinghamshire, England, UK',
 '1973 in\xa0Mexico',
 '1962 in\xa0Taunton, Somerset, UK',
 '1996 in\xa0Mexico, D.F., Mexico',
 '1965 in\xa0Georgetown, Guyana',
 '456 BC in\xa0Gela, Sicily, Italy',
 '1992 in\xa0Moscow, Russia',
 '1996 in\xa0Rome, Lazio, Italy',
 '1983 in\xa0Madrid, Spain',
 '1723 in\xa0Beja, Portugal',
 '1995 in\xa0Javea, Valencia, Spain',
 '1998 in\xa0Quezon City, Philippines \xa0(skin cancer)',
 '1992 in\xa0Istanbul, Turkey',
 '2001 in\xa0Istanbul, Turkey',
 '1994 in\xa0Moscow, Russia',
 '1997 in\xa0Philippines \xa0(cardiac arrest)',
 '1991 in\xa0Izmir, Turkey',
 '1959 in\xa0London, England, UK',
 '1975 in\xa0Belgrade, Yugoslavia',
 '1986 in\xa0Chiltern & Beaconsfield, Buckinghamshire, England, UK',
 '1994 in\xa0Greece',
 '1924 in\xa0France',
 '2009 in\xa0Lambeth, London, England, UK',
 '1998 in\xa0New York City, New York, USA',
 '

In [46]:
# If date_of_birth value is nan or length is >10 then defualt to NaN.
actors['date_of_death'] = [ np.nan if pd.isnull(x) or len(x)>10  else x for x in actors['date_of_death'] ]

In [47]:
# Verify Max length of the column is 10
actors['date_of_death'].dropna().str.len().max()

10

In [48]:
# Convert date of death column to datetime
# There is one date of death of value '1616-04-23'. Pandas only recognizes dates >= '1677-09-21' (pd.Timestamp.min).
# errors='coerce' changes it outbound date values to NaT
actors['date_of_death']=pd.to_datetime(actors['date_of_death'],format='%Y-%m-%d',errors = 'coerce')

In [49]:
# view columns and datatypes of names_csv
actors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297705 entries, 0 to 297704
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   imdb_name_id           297705 non-null  object        
 1   name                   297705 non-null  object        
 2   birth_name             297705 non-null  object        
 3   height                 44681 non-null   float64       
 4   bio                    204698 non-null  object        
 5   birth_details          110612 non-null  object        
 6   date_of_birth          101740 non-null  datetime64[ns]
 7   place_of_birth         103992 non-null  object        
 8   death_details          39933 non-null   object        
 9   date_of_death          39129 non-null   datetime64[ns]
 10  place_of_death         37038 non-null   object        
 11  reason_of_death        22694 non-null   object        
 12  spouses_string         45352 non-null   obje

In [50]:
#declarative Base
Base = declarative_base()

In [51]:
#Create Engine movies_db
engine = create_engine("sqlite:///movies_db.sqlite")

In [52]:
# This creates the DB (movies_dv.sqllite) and connects to it
conn = engine.connect()

In [53]:
# Define class i.e table to be created in DB
class Movies(Base):
    __tablename__='movies'
    imdb_title_id = Column(String, primary_key=True)
    title = Column(String)
    original_title = Column(String)
    year = Column(Integer)
    date_published = Column(Date)
    genre = Column(String)
    duration = Column(Integer)
    country = Column(String)
    language = Column(String)
    director = Column(String)
    writer = Column(String)
    production_company = Column(String)
    actors = Column(String)
    description = Column(String)
    imdb_ratings = Column(Float)
    votes = Column(Integer)
    budget = Column(String)
    usa_gross_income = Column(String)
    worldwide_gross_income = Column(String)

In [54]:
# Define class i.e table to be created in DB
class Ratings(Base):
    __tablename__='ratings'
    rating_id = Column(Integer, primary_key=True)
    title = Column(String)
    year = Column(Integer)
    imdb_ratings = Column(Float)
    tomatometer_rating = Column(Float)
    audience_rating = Column(Float)
    imdb_title_id = Column(String)
    rt_year = Column(Integer)

In [55]:
# Define class i.e table to be created in DB
class Movie_Names(Base):
    __tablename__='cast_and_crew'
    id = Column(Integer, primary_key=True)
    imdb_title_id = Column(String)
    ordering = Column(Integer)
    imdb_name_id = Column(String)
    category = Column(String)
    job = Column(String)
    characters = Column(String)

In [56]:
# Define class i.e table to be created in DB
class Names(Base):
    __tablename__='actors'
    imdb_name_id = Column(String,primary_key=True)
    name = Column(String)
    birth_name = Column(String)
    height = Column(Float)
    bio  = Column(String)
    birth_details  = Column(String)
    date_of_birth = Column(Date)
    place_of_birth  = Column(String)
    death_details  = Column(String)
    date_of_death = Column(Date)
    place_of_death  = Column(String)
    reason_of_death  = Column(String)
    spouses_string  = Column(String)
    spouses = Column(Integer)
    divorces = Column(Integer)
    spouses_with_children = Column(Integer)
    children = Column(Integer)

In [57]:
# Create the imdb_movie class
class imdb_movie(Base):
    __tablename__ = 'imdb_top_250_movies'
    title = Column(String, primary_key=True)
    movie_name = Column(String)

In [58]:
# Creates 'movies' and "imdb_top_250_movies" tables in DB
Base.metadata.create_all(engine)

In [59]:
# Inserts data from IMDB_movies to movies table in movies_db sqlite DB.
IMDB_movies.to_sql('movies',conn,index=False,if_exists='append')

In [60]:
# Inserts data from clean_ratings to ratings table in movies_db sqlite DB.
clean_ratings.to_sql('ratings',conn,index=False,if_exists='append')

In [61]:
# Inserts data from movie_names_csv to movie_names table in movies_db sqllite DB.
cast_and_crew.to_sql('cast_and_crew',conn,index=False,if_exists='append')

In [62]:
# Inserts data from names_csv to names table in movies_db sqllite DB.
actors.to_sql('actors',conn,index=False,if_exists='append')

In [63]:
# Close Connection
conn.close()

In [64]:
# To push the objects made and query the server we use a Session object
session = Session(bind=engine)

In [65]:
# create url variable
url="https://www.imdb.com/chart/top/?ref_=nv_mv_250"

In [66]:
# Retrieve page with the requests module
response = requests.get(url)

In [67]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [68]:
# Examine the results, then determine element that contains sought info
#print(soup.prettify())

In [69]:
# results are returned as an iterable list
top_250_results = soup.find_all('td', class_="titleColumn")

In [70]:
# Loop through returned results
for result in top_250_results:
    # Error handling
    try:
        # Identify and return title of listing
        title_name = result.find('a').text
        # Identify and return link to listing
        link = result.a['href']
        title_strings = link.split('/')
        

        # Print results only if title, price, and link are available
        if (title_name and link):
            print('-------------')
            print(title_name)
            print(link)
            print(title_strings[2])
            imdb_title_id = title_strings[2]
            # Create  instances of the imdb movie class
            movie = imdb_movie(title=imdb_title_id, movie_name=title_name)
            # Add these objects to the session
            session.add(movie)
                                   
    except AttributeError as e:
        print(e)

-------------
The Shawshank Redemption
/title/tt0111161/
tt0111161
-------------
The Godfather
/title/tt0068646/
tt0068646
-------------
The Godfather: Part II
/title/tt0071562/
tt0071562
-------------
The Dark Knight
/title/tt0468569/
tt0468569
-------------
12 Angry Men
/title/tt0050083/
tt0050083
-------------
Schindler's List
/title/tt0108052/
tt0108052
-------------
The Lord of the Rings: The Return of the King
/title/tt0167260/
tt0167260
-------------
Pulp Fiction
/title/tt0110912/
tt0110912
-------------
The Good, the Bad and the Ugly
/title/tt0060196/
tt0060196
-------------
The Lord of the Rings: The Fellowship of the Ring
/title/tt0120737/
tt0120737
-------------
Fight Club
/title/tt0137523/
tt0137523
-------------
Forrest Gump
/title/tt0109830/
tt0109830
-------------
Inception
/title/tt1375666/
tt1375666
-------------
The Lord of the Rings: The Two Towers
/title/tt0167261/
tt0167261
-------------
Star Wars: Episode V - The Empire Strikes Back
/title/tt0080684/
tt0080684
---

In [71]:
# push top 250 to imdb_top_250_movies table
session.commit()

In [72]:
# close session
session.close()